In [ ]:
%pip install install git+https://github.com/henrikbulldog/xdbutils.git

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/henrikbulldog/xdbutils.git to /tmp/pip-req-build-15_dh_xf
  Running command git clone --filter=blob:none --quiet https://github.com/henrikbulldog/xdbutils.git /tmp/pip-req-build-15_dh_xf
  Resolved https://github.com/henrikbulldog/xdbutils.git to commit bdbda5c38cbf85caa7e6dbf0ea14e7788e9760ec
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached install-1.3.5-py3-none-any.whl (3.2 kB)
  Using cached pyspark-3.4.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for xdbutils: filename=xdbutils-0.0.1-py3-none-any.whl size=20105 sha256=9e048b1625469506593f1c959c6b47d5d6ececd17a7fe84053e3f82c09e84fcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-9d02s4fv/wheels/1a/6a/3e/44dcde429c3ae21d23f89d6515608f65a63756dd3d7a19a8fb
Successfully built xdbutil

In [ ]:
from xdbutils import XDBUtils

xdbutils = XDBUtils(spark, dbutils)

pipeline = xdbutils.create_dlt_event_pipeline(
  source_system="dataplatform_eh_sandbox",
  source_class="clickstream",
  catalog="demo",
  tags={
    "data_owner": "Henrik Thomsen",
    "cost_center": "123456",
    "documentation": "https://github.com/henrikbulldog/xdbutils"
  },
  databricks_token=dbutils.secrets().get(scope="<scope>", key="<secret>")
)

In [ ]:
pipeline.event_to_bronze(
  eventhub_namespace="testeventhub",
  eventhub_name="testihubi2",
  eventhub_group_id="test-cg",
  eventhub_connection_string=dbutils.secrets().get(scope="<scope>", key="<secret>")
)

Name,Type
key,binary
value,binary
topic,string
partition,int
offset,bigint
timestamp,timestamp
timestampType,int
_ingest_time,timestamp
_quarantined,boolean


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
from pyspark.sql.functions import col, from_json

json_schema = StructType([  
    StructField("timeStamp", StringType()),
    StructField("ipAddress", StringType()),
    StructField("userId", StringType()),
    StructField("sessionId", StringType()),
    StructField("path", StringType()),
    StructField("queryParameters", StringType()),
    StructField("referrerUrl", StringType()),
    StructField("os", StringType()),
    StructField("browser", StringType()),
    StructField("timeToCompletePage", IntegerType()),
    StructField("eventFirstTimestamp", IntegerType()),
    StructField("eventDuration", IntegerType()),
    StructField("eventScore", IntegerType()),
    StructField("deviceType", StringType()),
    StructField("isLead", BooleanType()),
    StructField("diagnostics", StringType()),
    ])

pipeline.bronze_to_silver(
  parse=lambda df: (
    df
    .select(from_json(col("value").cast(StringType()), json_schema).alias("payload"))
    .select("payload.*")
    .where(col("timestamp").isNotNull())
  )
)

Name,Type
timeStamp,string
ipAddress,string
userId,string
sessionId,string
path,string
queryParameters,string
referrerUrl,string
os,string
browser,string
timeToCompletePage,int
